# Partie Thermique

---

In [1]:
from numpy import linspace, zeros
from numpy import linalg
from numpy import ndarray
import plotly.express as px
from math import nan, isnan

In [2]:
#Initialisation des constante

lbda = 0.96
rho = 2550
Cp = 840

N = 30
R = 0.01

t = 10
Nt = 1000

Text = linspace(1770,1670, Nt)
Tref = 1770

In [3]:
#Calcule des constantes
W = lbda/(rho * Cp)
dr = R / N
dt = t / Nt

T = []

#Initialisation de la temperaure
for i in range(Nt):
    T.append( ([Tref]*N)  + [Text[i]] )



def sq(x):
    return x*x

In [4]:

# for m in range(1, Nt):

#     T[m][0] = W * dt * (T[m-1][1] - T[m-1][0])/sq(dr) + T[m-1][0]

#     for i in range(1,N):
#         T[m][i] = W * dt * ( (T[m-1][i+1] - 2*T[m-1][i] + T[m-1][i-1])/sq(dr) + (4*(T[m-1][i+1] - T[m-1][i])) / ( sq(dr) * (2*i - 1))) + T[m-1][i]

In [5]:
#Tim = [ [ (T[m][i] + T[m][i-1])/2 for i in range(1, N+0)] for m in range(0, Nt)]
# totalDT = [ [T[m][i]-Tref for i in range(len(T[0])-1)] for m in range(len(T))]

In [6]:
# X = linspace(0, R, N)
# Y = T[-1][:-1]

# px.line(x = X , y = Y)

In [7]:
totalDT = []
X = linspace(0, R, N)
for i in range(Nt):
    totalDT.append([ -(r*r/(R*R) * 100) for r in X] )
#totalDT.append([0 for i in X])
DT = totalDT[0]
px.line(x = X, y = DT, title=r"Temperature en fonction de R")

# Partie Mécanique

---

In [8]:
p = 0

E = 71e9
nuc = 100
kc = E/(3*(1-2*nuc))
muc= E/(2*(1+nuc))


alpha_c=9e-6

In [9]:
def k(i, t) -> float:
    return kc

def mu(i, t) -> float:
    return muc

def nu(i, t) ->float:
    return nuc


def A(i, t) -> float:
    A = 4/(9*k(i)) + 1/(12 * mu(i))
    return A

def B(i, t) -> float:
    B = 1/(3*k(i))
    return B

def alpha(i, t) -> float:
    return alpha_c


## Construction de M et de R

In [10]:
sol_t = [[0]*(2*N)]

thorr = [ [0] * N for i in range(Nt)]
err = [ [0] * N for i in range(Nt)]


ajout_tot = [[]]


for temps in range (1, Nt):
    
    tho = nu(N-1, temps)/mu(N-1, temps)

    dttho = dt/tho if dt < tho else 0
    

    DT = totalDT[temps]

    M = [[0 for i in range(2*N)] for j in range(2*N)]
    Rmat = [0]*(2*N)

    M[0][1] = 1


    M[2*N-1][2*N-2] = -3*k(N, temps)
    M[2*N-1][2*N-1] = 4*mu(4, temps)/((dr*N)**3)
    Rmat[2*N-1] = p - 3*k(N, temps)*alpha(N, temps)*DT[N-1] - 2*mu(i, temps)*err[temps-1][N-2] + thorr[temps-1][N-2]*(1-dttho)



    for i in range(1, N):
        r = dr * i
        M[ 2*(i-1) + 1][ 2*(i-1) : 2*(i-1) + 4] = [-r, -1/sq(r), r, 1/sq(r)]

    for i in range(1,N):

        tho_prec = tho
        tho = nu(i, temps)/mu(i, temps)


        dttho = dt/tho if dt < tho else 0
        dttho_prec = dt/tho_prec if dt < tho_prec else 0


        r = dr*i
        M[2*(i-1)+2][2*(i-1) : 2*(i-1) + 4] = [-3*k(i, temps), 4*mu(i, temps)/(r**3), 3*k(i+1, temps), -4*mu(i+1, temps)/(r**3)]

        ajout_visco = - thorr[temps-1][i]*(1-dttho) + thorr[temps-1][i-1]*(1-dttho_prec) + 2*mu(i, temps)*err[temps-1][i] - 2*mu(i, temps)*err[temps-1][i-1]

        ajout_tot[-1].append(ajout_visco)


        if isnan(ajout_visco):
            print(thorr[temps-1][i-1])
            print(thorr[temps-1][i-2])
            print(err[temps-1][i-1])
            print(visco_prec)
            raise ValueError
        visco_prec = ajout_visco

        Rmat[2*(i-1)+2] = -3*k(i, temps)*alpha(i, temps)*DT[i]+3*k(i+1, temps)*alpha(i+1, temps)*DT[i-1] + ajout_visco

    
    sol = linalg.solve(M, Rmat)
    # if nan in sol :
    #     print(M)
    #     print(Rmat)
    #     raise ValueError
    sol_t.append(sol)

    ajout_tot.append([])

    thorr[temps] = [-4*mu(i, temps)*sol[2*i+1]/((dr*i)**3) for i in range(1,N)]
    err[temps] = [-2*sol[2*i+1]/((dr*i)**3) for i in range(1,N)]

In [11]:
def Tcontraintesi(sol_ab,i,r, temps) -> ndarray:
    '''Calcule de la contrainte a partir des solution
    Attention i>0'''

    sigmai=zeros((3,3))
    ai,bi=sol_ab[2*i],sol_ab[2*i+1]
    sigmai[0][0]= -4*mu(i, temps)*bi/(r**3) + 3*k(i, temps)*ai - 3*k(i, temps)*alpha(i, temps)*DT[i]
    sigmai[1][1]=  2*mu(i, temps)*bi/(r**3) + 3*k(i, temps)*ai - 3*k(i, temps)*alpha(i, temps)*DT[i]
    sigmai[2][2]=  2*mu(i, temps)*bi/(r**3) + 3*k(i, temps)*ai - 3*k(i, temps)*alpha(i, temps)*DT[i]

    return sigmai



def Tdeplacementsi(sol_ab,i,r) -> ndarray:
    '''Calcule du déplacement à partir des solution
    Attention i>0'''

    epsilonai=zeros((3,3))
    ai,bi=sol_ab[2*i],sol_ab[2*i+1]
    epsilonai[0][0]=ai - 2 * bi / (r**3)
    epsilonai[1][1]=ai + bi/(r**3)
    epsilonai[2][2]=ai + bi/(r**3)

    return epsilonai


def deplacement(sol_ab, i, r):
    ai,bi=sol_ab[2*i],sol_ab[2*i+1]
    return ai * r + bi/(r**2)

In [12]:
X = linspace(0, R, N)
Y1, Y2, Y3, Y4 = [], [], [], []


totalContrainte = []

for temps in range(0, Nt):
    #M = M_t[temps]
    sol = sol_t[temps]
    c = []
    for i in range(N):
        r = i*dr
        contrainte = Tcontraintesi(sol, i, r, temps)
        
        c.append(contrainte)

    totalContrainte.append(c)


#px.line(x = X, y = [Y1, Y2, Y3])
#px.line(x = X, y = Y4)

In [13]:
contrainte = totalContrainte[0]

Y1 = [contrainte[i][0][0] for i in range(len(contrainte))]
Y2 = [contrainte[i][1][1] for i in range(len(contrainte))]
Y3 = [contrainte[i][2][2] for i in range(len(contrainte))]


px.line(x = X, y = [Y1, Y2, Y3])